# Get score on new pipeline

## Import packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dyslexia.app import get_results
from dyslexia.ocr import extract_text_from_image
from dyslexia.io import load_image_from_string

import time
import gc
from IPython.display import display

import cv2
import numpy as np
import matplotlib.pyplot as plt
import jiwer
import pandas as pd
import pickle
import re

In [3]:
%env TESSDATA_PREFIX /usr/share/tesseract-ocr/4.00/tessdata/

env: TESSDATA_PREFIX=/usr/share/tesseract-ocr/4.00/tessdata/


## Score "Dictée" files

In [4]:
def compare_score(fpath, truth_path, show_txt=True):
    with open(truth_path, 'r') as f:
        truth = f.read()

    truth = re.sub(r'\n+', ' ', truth).strip()
    truth = re.sub(r' +', ' ', truth).strip()
    
    if show_txt:
        print('======== TRUTH =========')
        print(truth)
    
    res = get_results(fpath)
    
    if 'error' in res:
        print(res)
    else:
        txt = '\n'.join(res['paragraphs'])
        txt = re.sub(r'\n+', ' ', txt).strip()
        txt = re.sub(r' +', ' ', txt).strip()
        
        score = dict(jiwer.compute_measures(truth=truth, hypothesis=txt))
        score_algo = pd.Series(score).to_frame().T

        if show_txt:
            print()
            print('======== ALGO =========')
            print(txt)

        display(score_algo)
    
    
    img = load_image_from_string(fpath)
    txt = extract_text_from_image(img)
    txt = re.sub(r'\n+', ' ', txt).strip()
    txt = re.sub(r' +', ' ', txt).strip()
        
    if txt != '':
        if show_txt:
            print()
            print('======== ORIG =========')
            print(txt)
            
        score = dict(jiwer.compute_measures(truth=truth, hypothesis=txt))
        score_orig = pd.Series(score).to_frame().T
        display(score_orig)

In [5]:
dictee_file = '../Exemples/Dictée_ Le petit chaperon rouge.pdf'
dictee_truth_path = '../Exemples/truth/dictee.txt'

compare_score(dictee_file, dictee_truth_path)

======== TRUTH =========
Dictée: Le petit chaperon rouge La grand-mère du petit chaperon rouge était très malade. Elle demanda à sa petite-fille, avec qui elle vivait, de lui apporter quelque chose pour la guérir. Elle l'avertit, cependant, de faire attention au Grand Méchant Loup qui rodait dans les bois. Le loup faisait peut aux villageois des alentours depuis des années. Jusqu'à présent, aucun des chasseurs n'avait réussi à l'abattre. Le petit chaperon rouge se mit donc en route pour aider sa grand-mère. Elle redressa son capuchon et le plaça sur sa tête. Elle s'engouffra dans les bois rapidement. C'est là qu'elle entendit un bruit, Elle se retourna mais ne vit rien. Quand elle rentra avec le remède, sa grand-mère était au lit. Elle lui demanda d'approcher. Le petit chaperon rouge remarqua qu'elle avait de grandes dents et de grandes oreilles. Sa voix était bien plus grave. Elle se rendit alors compte que ce n'était pas sa grand-mère mais bien le loup qui s'était déguisé. Elle sorti

,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.071429,0.071429,0.137755,0.862245,182.0,14.0,0.0,0.0



======== ORIG =========
Dictée: Le petit chaperon rouge La grand-mère du petit chaperon rouge était très malade. Elle demanda à sa petite-fille, avec qui elle vivait, de lui apporter quelque chose pour la guérir. Elle l’avertit, cependant, de faire attention au Grand Méchant Loup qui rodait dans les bois. Le loup faisait peur aux villageois des alentours depuis des années. Jusqu’à présent, aucun des chasseurs n’avait réussi à l’abattre. Le petit chaperon rouge se mit donc en route pour aider sa grand-mère. Elle redressa son capuchon et le plaça sur sa tête. Elle s'engouffra dans les bois rapidement. C’est là qu’elle entendit un bruit. Elle se retourna mais ne vit rien. Quand elle rentra avec le remède, sa grand-mère était au lit. Elle lui demanda d’approcher. Le petit chaperon rouge remarqua qu’elle avait de grandes dents et de grandes oreilles. Sa voix était bien plus grave. Elle se rendit alors compte que ce n’était pas sa grand-mère mais bien le loup qui s’était déguisé. Elle sorti

,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.071429,0.071429,0.137755,0.862245,182.0,14.0,0.0,0.0


In [6]:
file = '../Exemples/Histoire-Geo.pdf'
truth_path = '../Exemples/truth/hist_geo.txt'

compare_df = compare_score(file, truth_path)

======== TRUTH =========
La Grèce Antique I. Le théâtre grec Les pièces de théâtre grecque étaient très appréciées dans l'Antiquité. Les Grecs des Temps Anciens appréciaient tout particulièrement les tragédies et le récit des épopées des héros. Les théâtres, où se déroulaients les représentations, avaient une forme particulière en demi-cercle: ce sont les amphithéâtres. Gradins Scène Organisation d'un amphithéâtre II. La société grecque Les grecs ont inventé la démocratie. Chacun est citoyen à partir de sa majorité et peut faire valoir son opinion sur la façon de diriger la cité.

======== ALGO =========
La Grèce Antique . Le théâtre grec Les pièces de théâtre grecque étaient très appréciées dans l’Antiquité. Les Grecs des Temps Anciens appréciaient tout particulièrement les tragédies et le récit des épopées des héros. Les théâtres, où se déroulaient les représentations, avaient une forme particulière en demi-cercle: ce sont les amphithéâtres. Gradins SS / — ; ( ) \ / Organisation d’un

,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.16092,0.147368,0.206171,0.793829,81.0,6.0,0.0,8.0



======== ORIG =========
La Grèce Antique . Le théâtre grec Les pièces de théâtre grecque étaient très appréciées dans l’Antiquité. Les Grecs des Temps Anciens appréciaient tout particulièrement les tragédies et le récit des épopées des héros. Les théâtres, où se déroulaient les représentations, avaient une forme particulière en demi-cercle: ce sont les amphithéâtres. Gradins S < ‘{/ — \“ \ / Organisation d’un amphithéâtre I. — La société grecque Les grecs ont inventé la démocratie. Chacun est citoyen à partir de sa majorité et peut faire valoir son opinion sur la façon de diriger la cité.


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.149425,0.138298,0.197726,0.802274,81.0,6.0,0.0,7.0


In [7]:
file = '../Exemples/SVT.pdf'
truth_path = '../Exemples/truth/svt.txt'

compare_df = compare_score(file, truth_path)

======== TRUTH =========
Les plantes 1. De quoi est composée la plante? La plante est composée: - d’une partie souterraine qui s’étend (parfois très loin de l’endroit où se trouve la plante en surface), - d’une partie en surface (tige, feuilles,graines,fruits,fleurs) 2. De quoi a besoin la plante? 2.1. Soleil Le soleil permet à la plante de récupérer de l’énergie pour se développer. Sans soleil, elle ne peut pas récolter de l’énergie pour survivre. 2.2. Oxygène Tout comme le soleil, l’oxygène est essentiel pour la survie de la plante. 2.3. Eau L’eau apporte des nutriments (qui sont essentiels à la plante pour rester en bon état). 2.4. Place La partie souterraine de la plante peut prendre beaucoup de place. Si les racines ne peuvent pas aller où elles veulent, la croissance de la plante peut être empêchée

======== ALGO =========
Les plantes 1. De quoi est composée la plante? La plante est composée: - d’une partie souterraine qui s'étend (parfois très loin de l’endroit où se trouve la p

,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.067164,0.066667,0.088634,0.911366,126.0,3.0,5.0,1.0



======== ORIG =========
Les plantes 1. De quoi est composée la plante? La plante est composée: - _ d’une partie souterraine qui s'étend (parfois très loin de l’endroit où se trouve la plante en surface), - _ d’une partie en surface (tige, feuilles,graines,fruits,fleurs) 2. De quoi a besoin la plante? 2.1. 2.2. Soleil Le soleil permet à la plante de récupérer de l’énergie pour se développer. Sans soleil, elle ne peut pas récolter de l’énergie pour survivre. Oxygène Tout comme le soleil, l'oxygène est essentiel pour la survie de la plante. L’eau apporte des nutriments (qui sont essentiels à la plante pour rester en bon état). La partie souterraine de la plante peut prendre beaucoup de place. Si les racines ne peuvent pas aller où elles veulent, la croissance de la plante peut être empêchée.


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.08209,0.080292,0.102442,0.897558,126.0,3.0,5.0,3.0


## Go further

In [8]:
from utils import load_all_files_path

subdir_paths, all_files = load_all_files_path()

In [9]:
dictee_files = [f for f in all_files if 'Dictee' in f]

In [10]:
truth_path = '../Exemples/truth/dictee.txt'

for file in dictee_files:
    print(file)
    compare_df = compare_score(file, truth_path, show_txt=False)
    print('==================================')

../Exemples/Dictee/IMG_20210329_122324.jpg


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.071429,0.071066,0.132679,0.867321,183.0,13.0,0.0,1.0


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,1.020408,1.0,1.0,0.0,0.0,196.0,0.0,4.0


../Exemples/Dictee/dictee_example_nb.jpg


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.5,0.497462,0.568922,0.431078,99.0,16.0,81.0,1.0


...

,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.5,0.497462,0.568922,0.431078,99.0,16.0,81.0,1.0


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.515306,0.510101,0.571383,0.428617,97.0,13.0,86.0,2.0


../Exemples/Dictee/IMG_20210329_122828.jpg


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.076531,0.076531,0.147204,0.852796,181.0,15.0,0.0,0.0


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.903061,0.903061,0.936488,0.063512,19.0,10.0,167.0,0.0


In [11]:
hist_geo_files = [f for f in all_files if 'Histoire_Geo' in f]

In [12]:
truth_path = '../Exemples/truth/hist_geo.txt'

for file in hist_geo_files:
    print(file)
    compare_df = compare_score(file, truth_path, show_txt=False)
    display(compare_df)
    print('==================================')

../Exemples/Histoire_Geo/IMG_20210329_122543.jpg


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.643678,0.491228,0.648485,0.351515,58.0,25.0,4.0,27.0


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.528736,0.516854,0.547811,0.452189,43.0,2.0,42.0,2.0


None

../Exemples/Histoire_Geo/IMG_20210329_123005.jpg


...

None

../Exemples/Histoire_Geo/IMG_20210329_122536.jpg


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.195402,0.173469,0.21444,0.78556,81.0,4.0,2.0,11.0


,wer,mer,wil,wip,hits,substitutions,deletions,insertions
0,0.321839,0.291667,0.409451,0.590549,68.0,13.0,6.0,9.0


None